# [Retrieval-based-Voice-Conversion-WebUI](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI) Training notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI_v2.ipynb)

In [ ]:
# @title #查看显卡
!nvidia-smi

In [ ]:
# @title 挂载谷歌云盘

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
%%bash
# ============ PREP PYTHON 3.10 ============
set -e
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest \
 | tar -xvj -C /usr/local/bin/ bin/micromamba --strip-components=1
/usr/local/bin/micromamba create -y -n rvc -c conda-forge python=3.10

cat >/usr/local/bin/pip310 <<'EOF'
#!/bin/bash
exec /usr/local/bin/micromamba run -n rvc python -m pip "$@"
EOF
chmod +x /usr/local/bin/pip310

cat >/usr/local/bin/py310 <<'EOF'
#!/bin/bash
exec /usr/local/bin/micromamba run -n rvc python "$@"
EOF
chmod +x /usr/local/bin/py310

/usr/local/bin/py310 - <<'PY'
import platform
print("Python:", platform.python_version())
PY


In [ ]:
# @title #安装依赖
!apt-get update -y
!apt-get -y install build-essential python3-dev ffmpeg

!pip310 install --upgrade setuptools wheel
!pip310 install "pip<24.1"

# Pre-requisitos que fairseq necesita con metadatos antiguos
!pip310 install "omegaconf==2.0.6" "hydra-core==1.0.7" "Cython<3" "setuptools<72"

# El resto tal y como marca la template, usando pip310
!pip310 install faiss-cpu==1.7.2 fairseq==0.12.2 gradio==3.14.0 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2


In [ ]:
# @title #克隆仓库

!mkdir Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!git init
!git remote add origin https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!git fetch origin cfd984812804ddc9247d65b14c82cd32e56c1133 --depth=1
!git reset --hard FETCH_HEAD

In [ ]:
# @title #更新仓库（一般无需执行）
!git pull

In [ ]:
# @title #安装aria2
!apt -y install -qq aria2

In [ ]:
# @title 下载底模

# v1
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G48k.pth

# v2
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D48k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G48k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D48k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G48k.pth

In [ ]:
# @title #下载人声分离模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

In [ ]:
# @title #下载hubert_base
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

In [ ]:
# @title #下载rmvpe模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o rmvpe.pt

# De OPUS a WAV procesado

In [ ]:
# OPUS -> WAV (mono, PCM16) con 3 MODOS: no_trim | soft_trim | vad
# === CONFIGURA AQUÍ ===
SRC  = "/content/drive/MyDrive/RAVC/fabiandataset"           # .opus
DST  = "/content/drive/MyDrive/RAVC/fabiandataset40k_clean"  # salida final
SR   = 40000                                                 # 48000 si quieres 48k
MODE = "no_trim"                                             # "no_trim" | "soft_trim" | "vad"

import os, glob, shlex, subprocess, sys, math
os.makedirs(DST, exist_ok=True)

def run(cmd):
    return subprocess.run(cmd, shell=True).returncode == 0

def ff(cmd_list):
    return subprocess.run(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# ——— helpers VAD (solo si MODE=="vad") ———
def vad_cut_to_wav(inp, out, sr=SR, aggressiveness=2):
    """
    Recorte por voz con webrtcvad: genera WAV temporal PCM16, trocea por voz y pega.
    Más robusto que silenceremove.
    """
    try:
        import webrtcvad, wave, contextlib, collections, struct
        import tempfile, numpy as np

        # 1) decodifica a wav mono/sr fijo
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpwav:
            tmp = tmpwav.name
        if not run(f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(inp)} -ac 1 -ar {sr} -sample_fmt s16 {shlex.quote(tmp)}'):
            return False

        # 2) lee frames y aplica VAD
        vad = webrtcvad.Vad(aggressiveness)
        frame_dur_ms = 30
        bytes_per_sample = 2
        frame_len = int(sr * frame_dur_ms / 1000) * bytes_per_sample
        with contextlib.closing(wave.open(tmp, 'rb')) as wf:
            assert wf.getnchannels() == 1 and wf.getsampwidth() == 2 and wf.getframerate() == sr
            pcm = wf.readframes(wf.getnframes())

        # trocear en frames
        frames = [pcm[i:i+frame_len] for i in range(0, len(pcm), frame_len)]
        voiced = [vad.is_speech(f, sr) if len(f)==frame_len else False for f in frames]

        # 3) agrupa regiones con voz
        chunks = []
        cur = []
        for i, (f,isv) in enumerate(zip(frames, voiced)):
            if isv:
                cur.append(f)
            else:
                if cur:
                    chunks.append(b"".join(cur)); cur=[]
        if cur:
            chunks.append(b"".join(cur))

        # si no hay voz, exporta 0.5s de silencio para no perder el archivo
        if not chunks:
            import numpy as np
            silence = (np.zeros(int(sr*0.5)).astype(np.int16)).tobytes()
            chunks = [silence]

        # 4) combina y escribe WAV temporal
        payload = b"".join(chunks)
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpcut:
            out_cut = tmpcut.name
        with wave.open(out_cut, 'wb') as wf:
            wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sr)
            wf.writeframes(payload)

        # 5) normaliza y escribe a destino final
        af = f"loudnorm=I=-20:LRA=7:TP=-1.5"
        ok = run(f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(out_cut)} -af {shlex.quote(af)} -ac 1 -ar {sr} -sample_fmt s16 {shlex.quote(out)}')
        try: os.remove(tmp)
        except: pass
        try: os.remove(out_cut)
        except: pass
        return ok
    except Exception as e:
        return False

# ——— procesamiento ———
patterns = ["**/*.opus","**/*.OPUS"]
paths = []
for pat in patterns:
    paths.extend(glob.glob(os.path.join(SRC, pat), recursive=True))
print("Encontrados .opus:", len(paths))
ok, fail = 0, 0

for p in paths:
    rel   = os.path.relpath(p, SRC)
    base  = os.path.splitext(rel)[0]
    out_dir = os.path.join(DST, os.path.dirname(rel)); os.makedirs(out_dir, exist_ok=True)
    out = os.path.join(out_dir, base + ".wav")

    if MODE == "no_trim":
        # SOLO normaliza, no toca duración
        af = "loudnorm=I=-20:LRA=7:TP=-1.5"
        cmd = f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(p)} -af {shlex.quote(af)} -ac 1 -ar {SR} -sample_fmt s16 {shlex.quote(out)}'
        ok += run(cmd); fail += (not ok or 0) and 0

    elif MODE == "soft_trim":
        # Recorte ULTRA conservador (+ normalización)
        # Umbral muy bajo (-50 dB) + exige 1.0 s de silencio para cortar
        af = (
          "silenceremove="
          "start_periods=1:start_silence=1.0:start_threshold=-50dB:"
          "stop_periods=1:stop_silence=1.0:stop_threshold=-50dB,"
          "loudnorm=I=-20:LRA=7:TP=-1.5"
        )
        cmd = f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(p)} -af {shlex.quote(af)} -ac 1 -ar {SR} -sample_fmt s16 {shlex.quote(out)}'
        ok += run(cmd); fail += (not ok or 0) and 0

    elif MODE == "vad":
        # Recorta por voz con WebRTC VAD + normaliza
        if not vad_cut_to_wav(p, out, sr=SR, aggressiveness=2):
            fail += 1
        else:
            ok += 1
    else:
        print(f"❌ MODE inválido: {MODE}"); break

print(f"Terminado. OK={ok} | Fail={len(paths)-ok} | Salida={DST}")


In [ ]:
# ===== AUDITORÍA EN CONSOLA: OPUS vs WAV (sin CSV) =====
SRC = "/content/drive/MyDrive/RAVC/fabiandataset"           # .opus originales
DST = "/content/drive/MyDrive/RAVC/fabiandataset40k_clean"  # .wav generados (no_trim)

import os, glob, subprocess, math, statistics

def ffprobe_duration(path: str):
    try:
        out = subprocess.check_output(
            ["ffprobe","-v","error",
             "-show_entries","format=duration",
             "-of","default=noprint_wrappers=1:nokey=1",
             path],
            stderr=subprocess.STDOUT, text=True
        ).strip()
        return float(out) if out and out != "N/A" else None
    except Exception:
        return None

def key_rel(path, root):  # relativa sin extensión
    return os.path.splitext(os.path.relpath(path, root))[0]

def fmt_hms(sec):
    if sec is None or not (isinstance(sec,(int,float)) and math.isfinite(sec)): return "N/A"
    sec = int(round(sec)); h = sec//3600; m = (sec%3600)//60; s = sec%60
    return f"{h:02d}:{m:02d}:{s:02d}"

# Recolecta OPUS
opus = []
for pat in ("**/*.opus","**/*.OPUS"): opus += glob.glob(os.path.join(SRC, pat), recursive=True)
if not opus:
    print(f"❌ No se encontraron .opus en: {SRC}")
else:
    rows = []
    for op in opus:
        key = key_rel(op, SRC)
        wav = os.path.join(DST, key + ".wav")
        src_sec = ffprobe_duration(op)
        if not os.path.exists(wav):
            rows.append({"key":key,"src_sec":src_sec,"dst_sec":None,"ratio":None,"status":"missing_wav"})
            continue
        dst_sec = ffprobe_duration(wav)
        ratio = (dst_sec/src_sec) if (src_sec and dst_sec and src_sec>0) else None
        rows.append({"key":key,"src_sec":src_sec,"dst_sec":dst_sec,"ratio":ratio,"status":"ok" if ratio is not None else "bad_meta"})

    total = len(rows)
    ok = sum(1 for r in rows if r["status"]=="ok")
    miss = sum(1 for r in rows if r["status"]=="missing_wav")
    bad = sum(1 for r in rows if r["status"]=="bad_meta")
    ratios = [r["ratio"] for r in rows if r["ratio"] is not None]
    total_src = sum((r["src_sec"] or 0) for r in rows)
    total_dst = sum((r["dst_sec"] or 0) for r in rows)
    mean_ratio = statistics.fmean(ratios) if ratios else None
    median_ratio = statistics.median(ratios) if ratios else None

    b_lt02 = sum(1 for r in ratios if r < 0.2)
    b_02_05 = sum(1 for r in ratios if 0.2 <= r < 0.5)
    b_05_08 = sum(1 for r in ratios if 0.5 <= r < 0.8)
    b_ge08 = sum(1 for r in ratios if r >= 0.8)

    worst = sorted((r for r in rows if r["ratio"] is not None), key=lambda x: x["ratio"])[:5]
    good  = sorted((r for r in rows if r.get("ratio") and r["ratio"]>=0.95), key=lambda x: -x["ratio"])[:5]

    print("========== AUDITORÍA OPUS→WAV ==========")
    print(f"Archivos .opus          : {len(opus)}")
    print(f"Pairs medidos (ok)      : {ok}")
    print(f"WAV faltantes           : {miss}")
    print(f"Metadatos inválidos     : {bad}")
    print(f"Duración total OPUS     : {fmt_hms(total_src)} ({total_src/3600:.2f} h)")
    print(f"Duración total WAV      : {fmt_hms(total_dst)} ({total_dst/3600:.2f} h)")
    print(f"Ratio medio dst/src     : {mean_ratio:.3f}" if mean_ratio is not None else "Ratio medio dst/src : N/A")
    print(f"Ratio mediano dst/src   : {median_ratio:.3f}" if median_ratio is not None else "Ratio mediano dst/src: N/A")

    print("\nBuckets ratio dst/src:")
    print(f"  <0.20     : {b_lt02}")
    print(f"  0.20–0.50 : {b_02_05}")
    print(f"  0.50–0.80 : {b_05_08}")
    print(f"  ≥0.80     : {b_ge08}")

    print("\n-- Peores 5 ratios (recorte/sesgo más alto) --")
    for r in worst:
        print(f"ratio={r['ratio']:.3f} | src={fmt_hms(r['src_sec'])} -> dst={fmt_hms(r['dst_sec'])} | {r['key']}")

    if good:
        print("\n-- Ejemplos buenos (ratio≥0.95) --")
        for r in good:
            print(f"ratio={r['ratio']:.3f} | src={fmt_hms(r['src_sec'])} -> dst={fmt_hms(r['dst_sec'])} | {r['key']}")

    if b_02_05 + b_lt02 > 0:
        print("\n⚠️  Hay recortes fuertes. Usa conversion **sin trim** o **soft_trim/vad**.")
    else:
        print("\n✅ Sin recortes groseros detectados.")


In [ ]:
# ========= 1) VERIFICAR CARPETA EN DRIVE (Bash) =========
%%bash
set -e
ls -lah "/content/drive/MyDrive/RAVC/wavsfiltrados" | head -n 20 || {
  echo "❌ No existe: /content/drive/MyDrive/RAVC/wavsfiltrados"; exit 1; }

In [ ]:
# ========== 1) VERIFICAR CARPETA EN DRIVE (Bash) ==========
%%bash
set -e
ls -lah "/content/drive/MyDrive/RAVC/fabiandataset40k_clean" | head -n 20 || {
  echo "❌ No existe: /content/drive/MyDrive/RAVC/fabiandataset40k_clean"; exit 1; }


In [ ]:
# ========= 2) CREAR ZIP EN DRIVE (Bash) =========
%%bash
set -e
mkdir -p "/content/drive/MyDrive/dataset"
# comprimimos la carpeta completa a un zip dentro de 'dataset'
cd "/content/drive/MyDrive"
zip -r -9 -q "dataset/midataset_wavsfiltrados.zip" "RAVC/wavsfiltrados"
# mostrar tamaño del zip resultante
ls -lh "/content/drive/MyDrive/dataset/midataset_wavsfiltrados.zip"

In [ ]:
# ========== 2) CREAR ZIP EN DRIVE (Bash) ==========
%%bash
set -e
mkdir -p "/content/drive/MyDrive/dataset"
cd "/content/drive/MyDrive"
# Comprime preservando la ruta relativa RAVC/fabiandataset40k_clean dentro del zip
zip -r -9 -q "dataset/fabian40k_clean.zip" "RAVC/fabiandataset40k_clean"
echo "✅ ZIP creado:"
ls -lh "/content/drive/MyDrive/dataset/fabian40k_clean.zip"


In [ ]:
# @title 从谷歌云盘加载打包好的数据集到/content/dataset
DATASET = "/content/drive/MyDrive/dataset/midataset_wavsfiltrados.zip"  # <-- TU ZIP

!mkdir -p /content/dataset
!unzip -o -d /content/dataset -B "{DATASET}"

In [ ]:
# ========== 3) CARGAR ZIP A /content/dataset (Python) ==========
DATASET = "/content/drive/MyDrive/dataset/fabian40k_clean.zip"  # <-- TU ZIP

!mkdir -p /content/dataset
# Descomprime preservando RAVC/fabiandataset40k_clean
!unzip -o -d /content/dataset -B {DATASET}

# Comprobación rápida del destino esperado:
!ls -lah "/content/dataset/RAVC/fabiandataset40k_clean" | head -n 20


In [ ]:
# @title #重命名数据集中的重名文件  (RENOMBRAR DUPLICADOS)
# Lista tu carpeta real
!ls -a "/content/dataset/RAVC/wavsfiltrados"

# Instala 'rename' si no está, y renombra archivos tipo "nombre.ext~N" -> "nombre_N.ext"
!apt-get update -y >/dev/null 2>&1 && apt-get install -y rename >/dev/null 2>&1
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/RAVC/wavsfiltrados/*.*~* || true


In [ ]:
# ========== 4) RENOMBRAR DUPLICADOS Y LISTAR (Python) ==========
# Instala 'rename' si no está, y renombra archivos tipo "nombre.ext~N" -> "nombre_N.ext"
!apt-get update -y >/dev/null 2>&1 && apt-get install -y rename >/dev/null 2>&1
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/RAVC/fabiandataset40k_clean/*.*~* || true

# Lista final (muestra primeras 30 entradas)
!ls -lah "/content/dataset/RAVC/fabiandataset40k_clean" | sed -n '1,30p'


# Instalar torchaudio


In [ ]:
# Desinstala lo que haya en py310 (por si acaso)
!pip310 uninstall -y torch torchaudio

# Instala torch==2.5.1 y torchaudio==2.5.1 para CUDA 12.1 (compatible con tu driver CUDA 12.x)
!pip310 install --index-url https://download.pytorch.org/whl/cu121 torch==2.5.1+cu121 torchaudio==2.5.1+cu121

# Verifica que es en py310 y con las versiones correctas
!py310 - <<'PY'
import platform, torch, torchaudio
print("Python:", platform.python_version())
print("torch:", torch.__version__, "| cuda:", torch.version.cuda)
print("torchaudio:", torchaudio.__version__)
PY


In [ ]:
!py310 -c "import platform, torch, torchaudio; print('Python:', platform.python_version()); print('torch:', torch.__version__, '| cuda:', torch.version.cuda); print('torchaudio:', torchaudio.__version__)"


# Instalar tensorboard

In [ ]:
# Instalar en el entorno correcto (py310)
!pip310 install -q tensorboard tensorboardX

# Verificación rápida
!py310 -c "import tensorboard, torch; print('tensorboard OK | torch', torch.__version__)"

# (opcional) comprobar que el índice existe
!test -f /content/Retrieval-based-Voice-Conversion-WebUI/logs/primer-test/added.index && echo 'index OK' || echo 'falta index'


# Instalar matplot

In [ ]:
# Instala versión compatible en el entorno correcto
!pip310 install -q "matplotlib<3.9"

# Verifica
!py310 -c "import matplotlib; print('matplotlib', matplotlib.__version__)"


# WEBUI

In [ ]:
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!env MPLBACKEND=Agg py310 infer-web.py --colab --pycmd py310 --port 7865 --noautoopen



In [ ]:
%%bash
EXP="fabian-test"   # <-- usa el nombre exacto de tu experimento
BASE="/content/Retrieval-based-Voice-Conversion-WebUI/logs/$EXP"
echo "=== Índices FAISS ==="
ls -lh "$BASE"/*.index 2>/dev/null || echo "⏳ Aún no hay .index"


In [ ]:
%%bash
EXP="fabian-test"
BASE="/content/Retrieval-based-Voice-Conversion-WebUI/logs/$EXP"

echo "=== ARCHIVOS DEL EXPERIMENTO ==="
ls -lh "$BASE" || { echo "❌ No existe $BASE"; exit 1; }

echo "=== ÍNDICES FAISS ==="
ls -lh "$BASE"/*.index 2>/dev/null || echo "⏳ Aún no hay .index"

echo "=== FEATURES DISCRETAS ==="
ls -lh "$BASE/3_feature256" | sed -n '1,15p' || true

echo "=== ESPACIO DISCO ==="
df -h | sed -n '1,5p'


In [ ]:
EXP = "fabian-test"
base = f"/content/Retrieval-based-Voice-Conversion-WebUI/logs/{EXP}"

import os, glob, numpy as np, re, math

# total_fea.npy si existe (consolidado)
p_total = os.path.join(base, "total_fea.npy")
if os.path.exists(p_total):
    X = np.load(p_total, mmap_mode="r")
    n_total = X.shape[0]
    dim = X.shape[1]
    print("total_fea.npy:", X.shape, "| aprox MB:", X.nbytes/1e6)
else:
    # suma de todos los .npy en 3_feature256
    files = sorted(glob.glob(os.path.join(base, "3_feature256", "*.npy")))
    n_total = 0
    dim = None
    for i,f in enumerate(files[:10]):  # mira 10 para inferir dim
        arr = np.load(f, mmap_mode="r")
        n_total += arr.shape[0]
        dim = arr.shape[1] if dim is None else dim
    # estimación para todos
    if len(files) > 10:
        # estima con la media de los 10 primeros
        avg = n_total/10
        n_total = int(avg*len(files))
    print(f"3_feature256: ficheros={len(files)} | estimación vectores={n_total} | dim≈{dim}")

# extrae nlist del nombre del índice entrenado
idxs = sorted(glob.glob(os.path.join(base, "trained_IVF*_v1.index")))
if idxs:
    m = re.search(r"IVF(\d+)_", os.path.basename(idxs[0]))
    nlist = int(m.group(1)) if m else None
    print("trained index:", os.path.basename(idxs[0]), "| nlist:", nlist)
    if nlist and n_total:
        print(f"N/nlist ≈ {n_total}/{nlist} ≈ {n_total/max(1,nlist):.1f} vectores por lista")
else:
    print("⚠️ No se encontró trained_IVF...index (raro si antes lo viste).")


In [ ]:
%%bash
echo "=== vCPU disponibles ==="; nproc
echo "=== Variables de hilos (si la WebUI las heredó) ==="
echo "OMP_NUM_THREADS=$OMP_NUM_THREADS"
echo "MKL_NUM_THREADS=$MKL_NUM_THREADS"
echo "=== Proceso Python más pesado ahora ==="
ps -eo pid,ppid,comm,%cpu,%mem --sort=-%cpu | head -n 10


In [ ]:
# Celda A (Python)
!py310 -c "import sys; print('Python', sys.version)"
!py310 -m pip uninstall -y gradio httpx httpcore anyio
!py310 -m pip install -q "gradio==3.14.0" "httpx==0.24.1" "httpcore==0.17.3" "anyio<4"
!py310 - <<'PY'
import sys, gradio, httpx, httpcore, anyio
print("py:", sys.version.split()[0])
print("gradio:", gradio.__version__)
print("httpx:", httpx.__version__)
print("httpcore:", httpcore.__version__)
print("anyio:", getattr(anyio, "__version__", "n/a"))
PY


In [ ]:
%%bash
pkill -f "infer-web.py" 2>/dev/null || true
fuser -k 7860/tcp 2>/dev/null || true
fuser -k 7861/tcp 2>/dev/null || true
echo "OK: WebUI parada y puertos libres."


In [ ]:
# Celda C (Python): esta celda queda ocupada y aquí verás el enlace
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!env GRADIO_QUEUE_ENABLED=0 MPLBACKEND=Agg py310 infer-web.py --colab --pycmd py310 --port 7860 --noautoopen --noparallel


In [ ]:
%%bash
EXP="fabian-test"
BASE="/content/Retrieval-based-Voice-Conversion-WebUI/logs/$EXP"

echo "== Carpeta de experimento =="
ls -ld "$BASE" || exit 0

echo -e "\n== Subcarpetas clave (deben existir y tener archivos) =="
for d in 0_gt_wavs 1_16k_wavs 2a_f0 2b-f0nsf 3_feature256; do
  printf "%-12s " "$d"
  test -d "$BASE/$d" && find "$BASE/$d" -type f | wc -l || echo "0"
done

echo -e "\n== total_fea.npy (features HuBERT) =="
if [ -f "$BASE/total_fea.npy" ]; then
  ls -lh "$BASE/total_fea.npy"
else
  echo "FALTA: $BASE/total_fea.npy"
fi

echo -e "\n== Índices FAISS =="
ls -lh "$BASE"/trained_IVF*_v1.index 2>/dev/null || echo "Falta trained_*.index (necesitas Train feature index)"
ls -lh "$BASE"/added*.index           2>/dev/null || echo "Falta added*.index (pulsa Train feature index o créalo con la celda FAISS)"


# Guardado al Drive

In [ ]:
# ====== GUARDAR EN DRIVE (usa esta celda Python con "!" delante) ======
%cd /content/Retrieval-based-Voice-Conversion-WebUI

# 1) crear carpetas destino
!mkdir -p "/content/drive/MyDrive/RVC_models/primer-test/logs"
!mkdir -p "/content/drive/MyDrive/RVC_models/weights"

# 2) copiar el modelo ligero que usa la UI
!cp -u weights/primer-test.pth "/content/drive/MyDrive/RVC_models/weights/"

# 3) copiar logs del experimento (checkpoints, índices, config, features resumidos)
#    si rsync no está, se usa fallback con cp -ru
# intento con rsync
!rsync -ah --info=stats1 --exclude="eval" logs/primer-test/ "/content/drive/MyDrive/RVC_models/primer-test/logs/" || true
# fallback (si rsync no existía)
!cp -ru logs/primer-test "/content/drive/MyDrive/RVC_models/" || true

# 4) listar para comprobar
!ls -lh "/content/drive/MyDrive/RVC_models/weights"
!ls -lh "/content/drive/MyDrive/RVC_models/primer-test/logs" | sed -n '1,120p'


In [ ]:
%%bash
set -e
cd /content/Retrieval-based-Voice-Conversion-WebUI

DST_BASE="/content/drive/MyDrive/RVC_models/fabian-test"
mkdir -p "$DST_BASE/weights" "$DST_BASE/logs"

# 1) Modelo ligero que usa la WebUI
cp -u "weights/fabian-test.pth" "$DST_BASE/weights/"

# 2) Todo el experimento (configs, índices, features resumidos, ckpts finales…)
#    Si existe rsync, úsalo; si no, fallback a cp -ru
if command -v rsync >/dev/null 2>&1; then
  rsync -ah --info=stats1 --exclude="eval" "logs/fabian-test/" "$DST_BASE/logs/"
else
  cp -ru "logs/fabian-test" "$DST_BASE/"
fi

echo "=== GUARDADO EN DRIVE ==="
du -sh "$DST_BASE"/weights/* 2>/dev/null || true
du -sh "$DST_BASE"/logs 2>/dev/null || true
ls -lh "$DST_BASE/weights" || true
ls -lh "$DST_BASE/logs" | sed -n '1,120p' || true


In [ ]:
%%bash
BASE="/content/drive/MyDrive/RVC_models/fabian-test"
echo "weights:"
ls -lh "$BASE/weights"
echo
echo "logs (mira que estén los índices *.index y config.json):"
ls -lh "$BASE/logs" | sed -n '1,120p'


# Cargar del Drive una vez se haya iniciado el drive y se haya apuntado al repo

In [ ]:
!mkdir -p weights logs
!rm -f weights/primer-test.pth
!ln -s "/content/drive/MyDrive/RVC_models/weights/primer-test.pth" weights/primer-test.pth

!rm -rf logs/primer-test
!ln -s "/content/drive/MyDrive/RVC_models/primer-test/logs" logs/primer-test

# asegúrate de tener alias corto del índice:
!bash -lc 'cd logs/primer-test && { test -f added.index || ln -sfn added_IVF*_*_v1.index added.index; }'


In [ ]:
%cd /content/Retrieval-based-Voice-Conversion-WebUI/logs/primer-test
# Si existe 'trained_*.index', apúntalo a added.index
!ln -sfn trained_IVF742_Flat_nprobe_1_v1.index added.index || true
# (alternativa) enlazar el "added_*" al nombre corto
!test -f added_IVF742_Flat_nprobe_1_v1.index && ln -sfn added_IVF742_Flat_nprobe_1_v1.index added.index || true
!ls -lh


CON FABIAN
OPCION a

In [ ]:
%%bash
cd /content/Retrieval-based-Voice-Conversion-WebUI

# 1) estructura local
mkdir -p weights logs

# 2) enlazar el .pth desde Drive
rm -f weights/fabian-test.pth
ln -s "/content/drive/MyDrive/RVC_models/fabian-test/weights/fabian-test.pth" weights/fabian-test.pth

# 3) enlazar los logs (índices, config, etc.)
rm -rf logs/fabian-test
ln -s "/content/drive/MyDrive/RVC_models/fabian-test/logs" logs/fabian-test

# 4) alias 'added.index' -> el índice entrenado (si no existe aún)
cd logs/fabian-test
{ test -f added.index || ln -sfn added_IVF*_*_v1.index added.index; }
ls -lh


OPCION B

In [ ]:
%%bash
cd /content/Retrieval-based-Voice-Conversion-WebUI

# Copia el .pth
mkdir -p weights
cp -u "/content/drive/MyDrive/RVC_models/fabian-test/weights/fabian-test.pth" weights/

# Copia los logs
mkdir -p logs/fabian-test
rsync -ah --info=stats1 "/content/drive/MyDrive/RVC_models/fabian-test/logs/" "logs/fabian-test/" || cp -ru "/content/drive/MyDrive/RVC_models/fabian-test/logs/"* "logs/fabian-test/"

# Asegura alias del índice
cd logs/fabian-test
{ test -f added.index || ln -sfn added_IVF*_*_v1.index added.index; }
ls -lh


# De OPUS a WAV INFERENCIA


In [ ]:
# OPUS -> WAV (mono, PCM16) con 3 MODOS: no_trim | soft_trim | vad
# === CONFIGURA AQUÍ ===
SRC  = "/content/drive/MyDrive/RAVC/Inferencia/fabian/malos"           # .opus
DST  = "/content/drive/MyDrive/RAVC/Inferencia/fabian/buenos"  # salida final
SR   = 40000                                                 # 48000 si quieres 48k
MODE = "no_trim"                                             # "no_trim" | "soft_trim" | "vad"

import os, glob, shlex, subprocess, sys, math
os.makedirs(DST, exist_ok=True)

def run(cmd):
    return subprocess.run(cmd, shell=True).returncode == 0

def ff(cmd_list):
    return subprocess.run(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# ——— helpers VAD (solo si MODE=="vad") ———
def vad_cut_to_wav(inp, out, sr=SR, aggressiveness=2):
    """
    Recorte por voz con webrtcvad: genera WAV temporal PCM16, trocea por voz y pega.
    Más robusto que silenceremove.
    """
    try:
        import webrtcvad, wave, contextlib, collections, struct
        import tempfile, numpy as np

        # 1) decodifica a wav mono/sr fijo
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpwav:
            tmp = tmpwav.name
        if not run(f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(inp)} -ac 1 -ar {sr} -sample_fmt s16 {shlex.quote(tmp)}'):
            return False

        # 2) lee frames y aplica VAD
        vad = webrtcvad.Vad(aggressiveness)
        frame_dur_ms = 30
        bytes_per_sample = 2
        frame_len = int(sr * frame_dur_ms / 1000) * bytes_per_sample
        with contextlib.closing(wave.open(tmp, 'rb')) as wf:
            assert wf.getnchannels() == 1 and wf.getsampwidth() == 2 and wf.getframerate() == sr
            pcm = wf.readframes(wf.getnframes())

        # trocear en frames
        frames = [pcm[i:i+frame_len] for i in range(0, len(pcm), frame_len)]
        voiced = [vad.is_speech(f, sr) if len(f)==frame_len else False for f in frames]

        # 3) agrupa regiones con voz
        chunks = []
        cur = []
        for i, (f,isv) in enumerate(zip(frames, voiced)):
            if isv:
                cur.append(f)
            else:
                if cur:
                    chunks.append(b"".join(cur)); cur=[]
        if cur:
            chunks.append(b"".join(cur))

        # si no hay voz, exporta 0.5s de silencio para no perder el archivo
        if not chunks:
            import numpy as np
            silence = (np.zeros(int(sr*0.5)).astype(np.int16)).tobytes()
            chunks = [silence]

        # 4) combina y escribe WAV temporal
        payload = b"".join(chunks)
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpcut:
            out_cut = tmpcut.name
        with wave.open(out_cut, 'wb') as wf:
            wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sr)
            wf.writeframes(payload)

        # 5) normaliza y escribe a destino final
        af = f"loudnorm=I=-20:LRA=7:TP=-1.5"
        ok = run(f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(out_cut)} -af {shlex.quote(af)} -ac 1 -ar {sr} -sample_fmt s16 {shlex.quote(out)}')
        try: os.remove(tmp)
        except: pass
        try: os.remove(out_cut)
        except: pass
        return ok
    except Exception as e:
        return False

# ——— procesamiento ———
patterns = ["**/*.opus","**/*.OPUS"]
paths = []
for pat in patterns:
    paths.extend(glob.glob(os.path.join(SRC, pat), recursive=True))
print("Encontrados .opus:", len(paths))
ok, fail = 0, 0

for p in paths:
    rel   = os.path.relpath(p, SRC)
    base  = os.path.splitext(rel)[0]
    out_dir = os.path.join(DST, os.path.dirname(rel)); os.makedirs(out_dir, exist_ok=True)
    out = os.path.join(out_dir, base + ".wav")

    if MODE == "no_trim":
        # SOLO normaliza, no toca duración
        af = "loudnorm=I=-20:LRA=7:TP=-1.5"
        cmd = f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(p)} -af {shlex.quote(af)} -ac 1 -ar {SR} -sample_fmt s16 {shlex.quote(out)}'
        ok += run(cmd); fail += (not ok or 0) and 0

    elif MODE == "soft_trim":
        # Recorte ULTRA conservador (+ normalización)
        # Umbral muy bajo (-50 dB) + exige 1.0 s de silencio para cortar
        af = (
          "silenceremove="
          "start_periods=1:start_silence=1.0:start_threshold=-50dB:"
          "stop_periods=1:stop_silence=1.0:stop_threshold=-50dB,"
          "loudnorm=I=-20:LRA=7:TP=-1.5"
        )
        cmd = f'ffmpeg -nostdin -hide_banner -loglevel error -y -i {shlex.quote(p)} -af {shlex.quote(af)} -ac 1 -ar {SR} -sample_fmt s16 {shlex.quote(out)}'
        ok += run(cmd); fail += (not ok or 0) and 0

    elif MODE == "vad":
        # Recorta por voz con WebRTC VAD + normaliza
        if not vad_cut_to_wav(p, out, sr=SR, aggressiveness=2):
            fail += 1
        else:
            ok += 1
    else:
        print(f"❌ MODE inválido: {MODE}"); break

print(f"Terminado. OK={ok} | Fail={len(paths)-ok} | Salida={DST}")

In [ ]:
# @title #手动将训练后的模型文件备份到谷歌云盘
# @markdown #需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

# @markdown #模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown #模型epoch
MODELEPOCH = 9600  # @param {type:"integer"}

!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/drive/MyDrive/
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/drive/MyDrive/

!cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth

In [ ]:
# @title 从谷歌云盘恢复pth
# @markdown 需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 模型epoch
MODELEPOCH = 7500  # @param {type:"integer"}

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

!cp /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/*.index /content/
!cp /content/drive/MyDrive/*.npy /content/
!cp /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth

In [ ]:
# @title 手动预处理（不推荐）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 采样率
BITRATE = 48000  # @param {type:"integer"}
# @markdown 使用的进程数
THREADCOUNT = 8  # @param {type:"integer"}

!python3 trainset_preprocess_pipeline_print.py /content/dataset {BITRATE} {THREADCOUNT} logs/{MODELNAME} True

In [ ]:
# @title 手动提取特征（不推荐）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 使用的进程数
THREADCOUNT = 8  # @param {type:"integer"}
# @markdown 音高提取算法
ALGO = "harvest"  # @param {type:"string"}

!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}

!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME} True

In [ ]:
# @title 手动训练（不推荐）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 使用的GPU
USEGPU = "0"  # @param {type:"string"}
# @markdown 批大小
BATCHSIZE = 32  # @param {type:"integer"}
# @markdown 停止的epoch
MODELEPOCH = 3200  # @param {type:"integer"}
# @markdown 保存epoch间隔
EPOCHSAVE = 100  # @param {type:"integer"}
# @markdown 采样率
MODELSAMPLE = "48k"  # @param {type:"string"}
# @markdown 是否缓存训练集
CACHEDATA = 1  # @param {type:"integer"}
# @markdown 是否仅保存最新的ckpt文件
ONLYLATEST = 0  # @param {type:"integer"}

!python3 train_nsf_sim_cache_sid_load_pretrain.py -e lulu -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained/f0G{MODELSAMPLE}.pth -pd pretrained/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA}

In [ ]:
# @title 删除其它pth，只留选中的（慎点，仔细看代码）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 选中模型epoch
MODELEPOCH = 9600  # @param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*.pth

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

In [ ]:
# @title 清除项目下所有文件，只留选中的模型（慎点，仔细看代码）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 选中模型epoch
MODELEPOCH = 9600  # @param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm -rf /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}